# 데이터소스
- 스파크 핵심 데이터소스 
  - CSV
  - JSON
  - 파케이
  - ORC
  - JDBC/ODBC연결
  - 일반 텍스트 파일

- 핵심 서드파티 데이터소스
  - 카산드라
  - HBase
  - 몽고디비
  - AWS Redshift
  - XML

In [0]:
path='/FileStore/tables/all/*.csv'

## 데이터소스 API의 구조

### 읽기 API 구조
- 스파크에서 데이터를 읽을 땐 기본적으로 DataFrameReader를 사용함
- DataFrameReader는 SparkSession의 read 속성으로 접근
- DataFrameReader를 얻고 나서는 다음과 같은 값을 지정해야함
  - format: 포맷 지정(default: 파케이)
  - option: 데이터 읽는 방법 지정 ex)키-값 쌍dlaus option('key', 'value')
  - schema: 데이터 소스에서 스키마를 제공하거나 스키마 추론 기능을 사용하려고 할 때 지정

In [0]:
spark.read

Out[1]: <pyspark.sql.readwriter.DataFrameReader at 0x7f6b4e3e9f40>

In [0]:
df = spark.read.format('csv')\
.option('mode', 'FAILFAST')\
.option('inferSchema', 'true')\
.option('path', path)\
.load()

---
- 위 예시에서는 읽기 모드를 'failfast'로 지정함
- <strong>읽기 모드: 스파크가 형식에 맞지 않는 데이터를 만났을 때의 동작 방식을 지정하는 옵션</strong>
  - permissive(default): 오류 레코드의 모든 필드를 null로 설정하고 모든 오류 레코드를 _corrupt_record라는 문자열 컬럼에 기록
  - dropMalformed: 형식에 맞지 않는 레코드가 포함된 로우 제거
  - failFast: 형식에 맞지 않는 레코드를 만나면 즉시 종료

### 쓰기 API 구조
- 데이터 읽기와 매우 유사한데 DataFrameReader 대신 DataFrameWriter를 사용
- 데이터소스에 항상 데이터를 기록해야하므로 DataFrame의 write속성을 사용
  - DataFrame별로 DataFrameWriter에 접근
- DataFrameWriter를 얻고나서는 다음과 같은 값을 지정해야함
  - format
  - option
  - 파일 기반의 데이터소스만 해당
    - partitionBy
    - bucketBy
    - sortBy

In [0]:
df.write.format('csv')\
.option('mode', 'OVERWRITE')\
.option('dateFormat', 'yyyy-MM-dd')\
.option('path', '/FileStore/tables/temp/temp.csv')\
.save()

- 위 예시에서는 저장 모드를 'overwrite'로 지정함
- 저장 모드: 스파크가 지정된 위치에서 동일한 파일이 발견됐을 때의 동작 방식 지정
  - append: 해당 경로에 이미 존재하는 파일 목록에 결과 파일 추가
  - overwrite: 이미 존재하는 모든 데이터를 완전히 덮어씀
  - errorIfExists(default): 오류를 발생시키면서 쓰기 작업이 실패
  - ignore: 아무런 처리 X

## CSV 파일
- CSV(comma-separated values)는 콤마(,)로 구분된 값을 의미
- 각 줄이 단일 레코드
- 각 필드를 콤마로 구분하는 일반적인 텍스트 파일 포맷
- 해당 포맷이 다루기 까다로운 이유
  - 운영 환경에서는 어떤 내용, 어떤 구조로 되어 있는지 등 다양한 전제를 만들어낼 수 없기 때문
  - 그래서 <strong>CSV reader는 많은 수의 옵션</strong>을 제공
  - [링크](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html?highlight=csv#pyspark.sql.DataFrameReader.csv)

### CSV 파일 읽기

In [0]:
spark.read.format('csv')\
.option('header', 'true')\
.option('mode','FAILFAST')\
.load(path)

Out[18]: DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: string, InvoiceDate: string, UnitPrice: string, CustomerID: string, Country: string]

### CSV 파일 쓰기

In [0]:
df.write.format('csv').mode('overwrite').option('sep','\t').save('/FileStore/tables/temp/tsv_file.tsv')

-----
- 위처럼 csv파일을 읽어 들여 tsv파일로 내보는 처리도 간단

## JSON 파일
- 자바스크립트 객체 표기법(JavaScript Object Notation)
- 스파크에서는 JSON파일을 사용할 때 줄로 구분된 JSON을 기본적으로 사용
  - 큰 JSON 객체나 배열을 하나씩 가지고 있는 파일을 다루는 것과 대조적인 부분
  - multiLine옵션으로 바꿀 순 있음
  - 근데 줄로 구분된 방식이 더 안정적이라서 디폴트임
    - 구조화 되어 있음
    - 최소한의 기본 데이터 타입이 존재함
    
- JSON은 객체이므로 CSV보다 옵션 수가 적음
  - [링크](https://spark.apache.org/docs/2.4.0/api/python/pyspark.sql.html?highlight=json#pyspark.sql.DataFrameReader.json)